In [ ]:
from pathlib import Path

import quackosm as qosm
from lonboard import viz, basemap
from pooch import retrieve
import duckdb

### Download PBF file

In [ ]:
DOWNLOAD_URL = (
    "https://download.bbbike.org/osm/bbbike/Miami/Miami.osm.pbf"
)
PBF_PATH = retrieve(
    url=DOWNLOAD_URL,
    path=Path(".").resolve(),
    known_hash=None,
)
PBF_PATH

### Convert PBF to GeoParquet file

In [ ]:
result_file = qosm.convert_pbf_to_parquet(pbf_path=PBF_PATH)
result_file

In [ ]:
duckdb.install_extension("spatial")
duckdb.load_extension("spatial")
duckdb.read_parquet(str(result_file)).project(
    "* EXCLUDE (geometry), ST_GeomFromWKB(geometry) geometry"
)

### Convert PBF to GeoDataFrame

In [ ]:
result_gdf = qosm.convert_pbf_to_geodataframe(pbf_path=PBF_PATH)
result_gdf

In [ ]:
viz(
    result_gdf.geometry,
    # optional style
    scatterplot_kwargs=dict(get_fill_color="#5c53a5"),
    path_kwargs=dict(get_color="#eb7f86"),
    polygon_kwargs=dict(stroked=False, get_fill_color="#f3e79b"),
    map_kwargs=dict(basemap_style=basemap.CartoBasemap.VoyagerNoLabels),
)